In [234]:
import pandas as pd
import sys, os
import tensorflow as tf
import shape_helper
from tensorflow import keras
from tensorflow.keras import layers

In [238]:

#the following bits are basically just to deal with translations and rotations in a pretty efficient manner. Note the 
#translation+removal functions have no for loops so it's pretty quick 
#Let's make the goal shape into a bunch of 1's and 0's representing where we want pieces
def mattoones(goalshape1):
    for i in range(len(goalshape1)):
        for k in range(len(np.transpose(goalshape1))):
            if goalshape1[i][k]!='0.0' and goalshape1[i][k]!=0:
                goalshape1[i][k]=1
    return goalshape1
#Remove all rows/columns at edges with only 0's in order to standardize calculations 
def translator(matrix):
    sums=0;
    while(sum(matrix[0])==0):
        if (sum(matrix[0]==0)):
            matrix=np.delete(matrix,0,0)

    i=len(matrix)-1
    while(sums==0):
        sums=sum(matrix[i]);
        if(sums==0):
            matrix=np.delete(matrix,i,0)
        i=i-1
    return(matrix)

#(*use the above function but for columns too*)
def removal(matrixer):
    c=translator(matrixer);
    c=translator(np.transpose(c))
    return(np.transpose(c))
def sqdiff(m1,m2):
    return (sum(sum((m1-m2)*(m1-m2))))
#I think this lets us visualize chains as 1/0s instead of HP but it's not very useful 
def graphicchainno(chain):
    boundary=np.floor(np.sqrt(len(chain))*2);
    lattice=np.ndarray.tolist(np.zeros([int(boundary),int(boundary)]))
    for i in range(len(chain)):
        lattice[chain[i][0]][chain[i][1]]=1
    lattice=np.array(lattice)
    return(lattice)

In [509]:
x=1
model=keras.Sequential([
    #Entry block: create four masks, and run
    layers.Conv2D(filters=30,kernel_size=3,activation="relu",padding='same',input_shape=[15,15,1]),
    layers.MaxPool2D(),
    #Block 2
    layers.Conv2D(filters=16, kernel_size=3,activation="relu",padding='same'),
    layers.MaxPool2D(),

    #Third hidden layer
    layers.Conv2D(filters=28,kernel_size=3,activation="relu",padding='same'),
    #output 
    layers.Flatten(),
    layers.Dense(units=169,activation="relu"),
    layers.Dense(units=169,activation="sigmoid")
])
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_84 (Conv2D)          (None, 15, 15, 30)        300       
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 7, 7, 30)         0         
 g2D)                                                            
                                                                 
 conv2d_85 (Conv2D)          (None, 7, 7, 16)          4336      
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 3, 3, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_86 (Conv2D)          (None, 3, 3, 28)          4060      
                                                                 
 flatten_28 (Flatten)        (None, 252)             

1+1

In [508]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
targets=df['path']
startshape=df['shape_mapping']
x_train=[]
y_train=[]
validtargets=[]
validshapes=[]
arraysize=13
#create our array entries 
for i in range(len(startshape)):
    c=shape_helper.deserialize_path(df['path'][i])
    d=path_to_shape(c,arraysize)
    position=start_to_class(d[2],arraysize)
    targets[i]=position
    startshape[i]=mattoones(d[0])

#onehotencode position
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
    y_train.append(targets[i])

#onehottablepos=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    x_train.append(startshape[i])
   # for x in range(len(startshape[i])):
       # for y in range(len(startshape[i][1])):
           # if startshape[i][x][y]!=0:
                #onehottablepos[i][x*len(startshape[i]+y)]=1



for i in range(len(startshape)):
    prob=random.random()
    eve=len(startshape)
    if prob<.1:

        validshapes.append(startshape[eve-i])
        validtargets.append(targets[eve-i])
        np.delete(x_train,eve-i,0)
        np.delete(onehottable,eve-i,0)
        
#startshapes=tf.stack(startshape,np.float32)
onehottableval=np.zeros([len(validtargets),arraysize*arraysize])
for i in range(len(validtargets)):
    onehottableval[i][validtargets[i]]=1
    
trainx=tf.convert_to_tensor(x_train)
trainy=tf.convert_to_tensor(onehottable)
valx=tf.convert_to_tensor(validshapes)
valy=tf.convert_to_tensor(onehottableval)


#trainy=[tf.from_numpy(item).float() for item in startshape]



startpos ==> (0, -2)
startpos ==> (0, 1)
startpos ==> (0, -2)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, -1)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 

In [533]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/Sequences_rows (4).csv')
targets=df['path']
startshape=df['shape_mapping']
lengths=df['length']
tots=len(targets)

        
x_train=[]
y_train=[]
validtargets=[]
validshapes=[]
arraysize=13
#create our array entries 
for i in range(len(startshape)):
    c=shape_helper.deserialize_path(df['path'][i])
    d=path_to_shape(c,arraysize)
    position=start_to_class(d[2],arraysize)
    targets[i]=position
    startshape[i]=mattoones(d[0])

#onehotencode position
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
    y_train.append(targets[i])

#onehottablepos=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    x_train.append(startshape[i])
   # for x in range(len(startshape[i])):
       # for y in range(len(startshape[i][1])):
           # if startshape[i][x][y]!=0:
                #onehottablepos[i][x*len(startshape[i]+y)]=1



for i in range(len(startshape)):
    prob=random.random()
    eve=len(startshape)
    if prob<.1:

        validshapes.append(startshape[eve-i])
        validtargets.append(targets[eve-i])
        np.delete(x_train,eve-i,0)
        np.delete(onehottable,eve-i,0)
        
#startshapes=tf.stack(startshape,np.float32)
onehottableval=np.zeros([len(validtargets),arraysize*arraysize])
for i in range(len(validtargets)):
    onehottableval[i][validtargets[i]]=1
    
trainx=tf.convert_to_tensor(x_train)
trainy=tf.convert_to_tensor(onehottable)
valx=tf.convert_to_tensor(validshapes)
valy=tf.convert_to_tensor(onehottableval)


#trainy=[tf.from_numpy(item).float() for item in startshape]



startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -2)
startpos ==> (-2, 1)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-2

In [534]:
len(lengths)

3099

In [535]:
import math as math
def path_to_shape(path,grid_sizes):
    grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # actual grid for mapping
    temp_grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # temp grid to hold the array before alignment
    for coord in path:
        temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = 1 # path transferred onto grid but uncentered
    temp_grid[round(grid_sizes/2)][round(grid_sizes/2)] = 2  
    temp_grid = np.pad(temp_grid, 1) # zero padding to avoid multiplying by 0 when calculating moments

    # find centroid of temp_grid
    m_00 = len(path) # non-zero residues
    m_01 = 0
    for row_n, row in enumerate(temp_grid):
        if np.any(row!=0):
            m_01 += row_n * np.count_nonzero(row)
    m_10 = 0
    for col_n in range(np.shape(temp_grid)[1]):
        if np.any(temp_grid[:, col_n]!=0):
            m_10 += col_n *  np.count_nonzero(temp_grid[:, col_n])

    # coordinates of centroid
    n_centroid = math.floor((m_10/m_00))
    m_centroid = math.floor((m_01/m_00))
    centroid = (m_centroid, n_centroid)
    #print("centroid ==>", centroid)
    # align temp_grid onto grid
    dev_m = round(grid_sizes/2)-centroid[0]
    dev_n = round(grid_sizes/2)-centroid[1]
    coord_list = np.nonzero(temp_grid)
    print("startpos ==>", (dev_m,dev_n))
    start= (round(grid_sizes/2)+dev_m,round(grid_sizes/2)+dev_n)
    for i in range(len(coord_list[0])):
        grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 1
        #if i==0:
            #grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 2
    
    return temp_grid, grid,start
def start_to_class(coords,grid_size):
    return coords[0]*grid_size+coords[1]

In [536]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='MeanSquaredError',metrics=['AUC']
)

In [537]:
history = model.fit(
    trainx,trainy,epochs=20
)


Epoch 1/20
97/97 [==============================] - 1s 5ms/step - loss: 0.0056 - auc: 0.8148
Epoch 2/20
97/97 [==============================] - 1s 6ms/step - loss: 0.0056 - auc: 0.8165
Epoch 3/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0056 - auc: 0.8173
Epoch 4/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0056 - auc: 0.8180
Epoch 5/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0056 - auc: 0.8184
Epoch 6/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0056 - auc: 0.8189
Epoch 7/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0055 - auc: 0.8196
Epoch 8/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0055 - auc: 0.8201
Epoch 9/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0055 - auc: 0.8208
Epoch 10/20
97/97 [==============================] - 0s 5ms/step - loss: 0.0055 - auc: 0.8214
Epoch 11/20
97/97 [==============================] - 0s 5ms/step - lo

In [538]:
x=model.predict(valx)

10/10 [==============================] - 0s 2ms/step


In [539]:
thresx=threshold(x)
toterr=meanerror(thresx,valy)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [486]:
def meanerror(array1,array2):
    total=0
    for i in range(len(array1)):
        sum=0
        for j in range(len(array1[1])):
            sum=sum+math.sqrt((array1[i][j]-array2[i][j])**2)
        total=sum+total  
    return total
def threshold(array1):
    lop=np.zeros([len(array1),len(array1[1])])
    for i in range(len(array1)):
        lop[i][np.where(x[i]==max(x[i]))]=1
    print(lop)
    return lop



In [540]:
toterr/(len(valy)*2)

0.7722772277227723